In [1]:
import pandas as pd
import string
import re
import nltk
import time


stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()


df = pd.concat([pd.read_csv('data/data_{}.csv'.format(a)) for a in range(10)])
print(df.shape)
df = df.drop_duplicates()
print(df.shape)
df.head()


(9656817, 2)
(8961258, 2)


,Tweet,Category
0,i love lebron. http://bit.ly/PdHur,0
1,"@ludajuice Lebron is a Beast, but I'm still ch...",0
2,@wordwhizkid Lebron is a beast... nobody in th...,0
3,Bill Simmons in conversation with Malcolm Glad...,0
4,playing with Java and the Twitter API,0


In [2]:
# retweets?
df['Tweet'] = df['Tweet'].astype(str)
df['Retweet'] = df['Tweet'].apply(lambda x:x[:2]=='RT')

df['Retweet'].sum()

647511

In [3]:
import numpy as np
np.sum(df['Category'])

2365553

In [4]:

print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 8961258
Russian 2365553


In [5]:
def preprocess_text(series):
     # @somone to @
    lst = []
    for text in series:
         # remove all punctuation except @ and #
        text = ' '.join(['@' if '@' in word else word for word in text.split()])
        
        text = ' '.join(['#' if '#' in word else word for word in text.split()])
        text = ''.join([word.lower() for word in text if word not in string.punctuation.replace('@','').replace('#','')])


        # the real work, pos tagged only
        
        tokens = [a[0] if '@' in a[0] or '#' in a[0] or 'IN' in a[1] or 'N' not in a[1]
                    else a[1] for a in nltk.pos_tag(text.split())]
       
        lst.append(' '.join(tokens))
    return lst 

In [ ]:
# test it...
test = df.head().copy()
test['Processed'] = preprocess_text(test['Tweet'])
test.head()

,Tweet,Category,Retweet,Processed
0,i love lebron. http://bit.ly/PdHur,0,False,NN love NN NN
1,"@ludajuice Lebron is a Beast, but I'm still ch...",0,False,@ NN is a NN but NN still cheering 4 the NN th...
2,@wordwhizkid Lebron is a beast... nobody in th...,0,False,@ NN is a NN NN in the NN comes even close
3,Bill Simmons in conversation with Malcolm Glad...,0,False,NN NNS in NN with NN NN NN
4,playing with Java and the Twitter API,0,False,playing with NN and the NN NN


In [ ]:
import pickle
start = time.time()
df['Processed'] = preprocess_text(df['Tweet'])
end = time.time()
print('It took {} s to process/pos-tag the tweets.'.format(end-start))
pickle.dump( df, open( "files/df.pickle", "wb" ) )
df.head()